In [1]:
import praw
# import posixpath
# import urllib.parse
# import requests

In [2]:
reddit = praw.Reddit(client_id='YURuUMecfcUDaQ', client_secret='nOJD5agHsie1Z8QL1xRAeRO0rGA', user_agent='HypeOJ')

In [3]:
hot_posts = reddit.subreddit('Sneakers').hot(limit=10)
for post in hot_posts:
    print(post.title)

New Releases Thread 5/4 - 5/10
MMW x FOG
🎂 Nike Sacai x LD Waffle on my 24th birthday! 🎂
Reverse Skunk 69/420 has surfaced
Finally got the new laces in 😊
Graduated w my Grails ✌️👨‍🎓
Something special on foot today. Original 1989 Air jordan 4 Black Cement. A amazing shoe and one of my favorites.
Half-Cold Half-Hot
Michael Jordan - 1984 ROTY
Posted this in another sub to get enough karma to post here! Here's my current Jordan collection (with lots of room grow). Which one's your favorite?


In [25]:
import pandas as pd
posts = []
sneakers_subreddit = reddit.subreddit('Sneakers')
for post in sneakers_subreddit.hot(limit=50):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
posts.head(50)

,title,score,id,subreddit,url,num_comments,body,created
0,New Releases Thread 5/4 - 5/10,29,gd65b6,Sneakers,https://www.reddit.com/r/Sneakers/comments/gd6...,1605,"Please post all your questions, pics and comme...",1.588601e+09
1,MMW x FOG,3068,gfvpn1,Sneakers,https://i.redd.it/z10bs05jckx41.jpg,94,,1.588982e+09
2,🎂 Nike Sacai x LD Waffle on my 24th birthday! 🎂,690,gfv468,Sneakers,https://i.redd.it/a7llf5kl6kx41.jpg,64,,1.588980e+09
3,Reverse Skunk 69/420 has surfaced,361,gfyjq5,Sneakers,https://i.redd.it/4zt846453lx41.jpg,34,,1.588991e+09
4,Finally got the new laces in 😊,736,gftnqw,Sneakers,https://i.redd.it/qwp04xvzqjx41.jpg,33,,1.588975e+09
5,Graduated w my Grails ✌️👨‍🎓,254,gfxrrc,Sneakers,https://i.redd.it/o9tdn1a2wkx41.jpg,17,,1.588989e+09
6,Posted this in another sub to get enough karma...,62,gg4yuy,Sneakers,https://i.redd.it/ycrhu7cgtmx41.jpg,17,,1.589012e+09
7,Half-Cold Half-Hot,74,gg3zlm,Sneakers,https://i.redd.it/6uq7ja10jmx41.jpg,5,,1.589009e+09
8,Something special on foot today. Original 1989...,299,gfv8v4,Sneakers,https://i.redd.it/otabzd7y7kx41.jpg,35,,1.588981e+09
9,Michael Jordan - 1984 ROTY,129,gg005n,Sneakers,https://i.redd.it/m9ti2iifglx41.jpg,5,,1.588996e+09


In [7]:
import os
import urllib.request
# urllib.request.urlretrieve("https://i.redd.it/z10bs05jckx41.jpg", "00000001.jpg")

In [14]:
posts['url'][1]
len(posts['url'])

10

In [ ]:
# scope of the project: why is this important? could we use this to begin to see which brands and trends are on the come up?
# drop all non-jpg rows?
# how to link data to upvotes
# convert date column to a readable datetime format
# Brands to decipher: Nike, Jordan, Adidas, New_Balance
# See if I can scrape from a certain time frame
# ^ check to see if Reddit is OK with me querying this much data
# 

In [24]:
# iterate through all queried posts and download the corresponding image as a jpg file
# working on sending these images to the data folder

for i in range(1, len(posts['url'])):
    try:
        urllib.request.urlretrieve(posts['url'][i], "data/0000"+str(i)+".jpg")
    except:
        pass